# Lab 3: AI/BI Genie
In this lab, we will build an AI/BI Genie space to help us talk to our data using natural language!

AI/BI Genie is a no-code interface powered by AI with data intelligence that allows business users to conduct self-serve data analytics using natural language. Domain experts, such as data analysts, configure Genie spaces with datasets, sample queries, and text guidelines to help Genie translate business questions into analytical queries. After set up, business users can ask questions and generate visualizations to understand operational data. You can continuously update Genie’s semantic knowledge as your data changes and users pose new questions.

Link to the existing [Genie Space](https://e2-demo-field-eng.cloud.databricks.com/genie/rooms/01efea29f94016acbf0ed6dbf32cf77e?o=1444828305810485)

## Part 1: Creating the Genie Space

### 1.1: Navigate to the side panel menu --> SQL --> Genie. Click "New" to create a new Genie Space.
![](./Images/Create Genie Space.png)

### 1.2: Create the Genie Space by configuring the following fields:
* Title: [Your name] - Customer 360
* Description: This Genie Space contains information on customer, orders, and predictions on churn. 
* Default Warehouse: Select any available Serverless SQL Warehouse
* Tables: 
  * data_pioneers.c360.churn_orders
  * data_pioneers.c360.churn_prediction
  * data_pioneers.c360.churn_users
* Sample Questions: We'll come back later and add more, but for now, feel free to add:
> How many customers are predicted to churn?


### 1.3: Create a New Chat and start asking questions!
*Duration: 5 minutes*

At this point, Genie doesn't have much context on the datasets, in terms of semantics, definitions, calculations, etc. 

Let's play around with the Genie space for 5 minutes to get a baseline, and through the course of this workshop, we'll make Genie even smarter.

![](./Images/Genie New Chat.png)


### 1.4: Add General Instructions

In order to make Genie more intelligent, let's provide more context on important definitions and semantics of the data. One mechanism to do this is through the Genral Instructions tab.

Try this one as a starting point:
* A user's country should only be displayed to end users as the country's full name, not the code in the dataset. Use this mapping to link the code to the country full name.
  * FR --> France
  * SPAIN --> Spain
  * USA --> United States

#### Test it out! 

Let's ask a question and see how Genie can conceptualize the question based on the information we provided in the *General Instructions* configuration.

> Question: How many users are from France? 

> Note: After making changes to the *Instructions* tab, create a New Chat to apply the changes.


#### Other suggestions:

* Do not display the raw age group data (e.g. 0, 1, 2, etc.), only present the values below (18-25, 26-30, etc.). Use this mapping to link the raw age group field to the age group.
  * 0 --> 18-25
  * 1 --> 26-30
  * 2 --> 31-35
  * 3 --> 36-40
  * 4 --> 41-45
  * 5 --> 46-50
  * 6 --> 51-55
  * 7 --> 56-60
  * 8 --> 61-65
  * 9 --> 65-70
  * 10 --> 70+

### 1.5: Add Example SQL Queries

In addition to the text-based instructions above, we can also provide example SQL queries to enrich AI/BI Genie's understanding of the dataset. Let's add a few example queries and see how AI/BI Genie can further improve.

![](./Images/Genie Add Example Queries.png)


Let's add the following query...
```
SELECT COUNT(DISTINCT user_id) as count_customers
   , CASE churn_prediction
      WHEN 1 THEN 'Predicted to Churn'
      WHEN 0 THEN 'Predicted to Not Churn'
    END AS churn_prediction
FROM data_pioneers.c360.churn_prediction
GROUP BY ALL;
```

... which answers the question:
> How many customers are predicted to churn vs. not churn?


##### Additional Queries:

> Question: Which users are predicted to churn have the most historical orders?
```
WITH orders_by_user AS (
SELECT SUM(amount) as sum_total_amount
  , count(distinct order_id) as count_orders
  , max(creation_date) as last_transaction_date
  , user_id 
FROM churn_orders
GROUP BY ALL
)
SELECT c.user_id
  , o.sum_total_amount
  , o.count_orders
  , o.last_transaction_date
  , c.churn_prediction
FROM orders_by_user o
LEFT JOIN churn_prediction c ON o.user_id = c.user_id
WHERE churn_prediction = 1
ORDER BY sum_total_amount DESC
```


### 1.6: Add a Trusted Asset

Trusted assets are predefined functions and example queries meant to provide verified answers to questions that you anticipate from users. For certain datasets and calculations, it's important to provide users with the verification that the answer is 100% correct, and Trusted Assets provided the mechanism to do so.

To create a Trusted Asset, we need to first write our code as a Table-Valued Function. See below:

In [0]:
%sql
USE CATALOG data_pioneers;
USE SCHEMA c360;

CREATE OR REPLACE FUNCTION revenue_items_sold_per_day() RETURNS TABLE
    RETURN 
select sum(amount) as revenue_usd
  , sum(item_count) as items_sold
  , date(creation_date) as order_date
from data_pioneers.c360.churn_orders
group by all;

On the Instructions page, navigate to the "SQL Functions" section in the bottom left. 

![](./Images/Genie Navigate to SQL Function.png)

Add the function we just created as a Trusted Asset and click "Save".

![](./Images/Genie Add SQL Function_2.png)

Whenever a question about Revenue or Items Sold per day is asked, Genie will use this function to get the answer to the question. 

![](./Images/Genie Show Trusted Asset.png)


### 1.7: Provide Feedback and Monitor Results

We've built a pretty good start for our Genie Space, and now it's time to get feedback from users! Genie will get some things right, and it will get some things wrong. At this stage, it's an iterative process to continually refine it to handle the edge cases and ensure accurate results. 

Upon answering each question, users should indicate their feedback with a Thumbs Up, Thumbs Down, or Requesting a Review from the Genie Space Admin.

#### Thumbs Up!
![](./Images/Genie Feedback Thumbs Up.png)

#### Thumbs Down and Needs a Review
![](./Images/Genie Feedback Request a Review.png)


As the Genie Space administrator, we can view the History of all conversations and see the feedback that our users are providing. Depending on the feedback, we can make updates to the General Instructions, Example SQL Queries, and/or Trusted Assets to remediate the change. Now we've built a tight feeedback loop to get input from users, make changes, and have them try again.

![](./Images/Genie Feedback History.png)


### 1.8: Benchmarks

#### Create Benchmark
Benchmarks help us ensure that our Genie Space is continually providing accurate results, even as we're making changes to our General Instructions, Example SQL Queries, and Trusted Assets. Think about them as a Unit Test for your Genie Space.

Let's create a Benchmark to show how this works. Navigate to the "Benchmarks" menu on the side panel and click "Add Benchmark". Provide the following inputs:

> Question: How many users are from France?

> SQL Answer: 
```
SELECT
  COUNT(`user_id`) AS `user_count`
FROM
  `data_pioneers`.`c360`.`churn_users`
WHERE
  `country` = 'FR'
```

![](./Images/Genie Add Benchmark.png)

If you recall the above, in our General Instructions section, we indicated that the "FR" code corresponds to users from France. By adding this question to our Benchmarks, we can ensure that this question is consistently being answered correctly, as we add changes to our Genie Space configuration.


#### Run Benchmark

Now, let's go ahead and run the Benchmark and see how it works. After clicking "Run Benchmark", Genie will answer the `Question` specified in the Benchmark and compare the results to the `SQL Answer`. If the data matches, we're good to go! If the data doesn't match, we should compare the Genie-generated SQL with the `SQL Answer` and understand how they differ.

![](./Images/Genie Benchmark Results.png)

In this case, we're good to go! 

Now, let's try making a change to the General Instructions, and see how the Benchmark will respond to the change. 

For the sake of example, let's change the mapping for countries to say that the code "FR" actually corresponds to "Spain" and the code "SPAIN" corresponds with "France". Of course, this is incorrect, but this represents the type of change that may inadvertently be made to the Genie Space's Instructions that can result in negative results that we would want to test for.

Let's overwrite the General Instructions with the following:
* A user's country should only be displayed to end users as the country's full name, not the code in the dataset. Use this mapping to link the code to the country full name.
  * FR --> Spain
  * SPAIN --> France
  * USA --> United States

After making the change, let's Run Benchmark again and see the results.

![](./Images/Genie Benchmark Results Bad.png)

As expected, this Benchmark did not pass, as we got a different answer due to the change in the Instructions. 

To summarize, Benchmarks can help us test our Genie Spaces to ensure answers are continually correct, in light of changes to the Genie Space configuration.

Let's go ahead and revert the General Instructions to the correct mapping and move on to the next section.


### 1.9: Add Genie to Dashboard

Finally, Genie Spaces can either be accessed through the Genie UI, as we've showed during this workshop. Often times, the types of questions we would ask a Genie space are natural follow up questions we have after looking at data in a Dashboard. 

Using the Dashboard we build in the AI/BI Dashboards lab, let's attach this Genie space to the dashboard to provide a seamless experience!

![](./Images/Genie Add to Dash.png)

Once configured, we can ask questions of the Genie Space from the Dashboard UI!

![](./Images/Genie Query in Dash.png)


## Part 2: Best Practices and Recommendations

### Start small
  * Curating a Genie space is an iterative process. When creating a new space, start as small as possible, with minimal instructions and a limited set of questions to answer. Then, you can add as you iterate based on feedback and monitoring. This approach helps streamline creating and maintaining your space and allows you to curate it effectively in response to real user needs.

  * Use the following guidelines to help create a small Genie space:

    * Stay focused: Include only the tables necessary to answer the questions you want the space to handle. Aim for five or fewer tables. The more focused your selection, the better. Keeping your space narrowly focused on a small amount of data is ideal, so limit the number of columns in your included tables.
    * Plan to iterate: Start with a minimal setup for your space, focusing on essential tables and basic instructions. Add more detailed guidance and examples as you refine the space over time, rather than aiming for perfection initially.
    * Build on well-annotated tables: Genie uses Unity Catalog column names and descriptions to generate responses. Clear column names and descriptions help produce high-quality responses. Column descriptions should offer precise contextual information. Avoid ambiguous or unnecessary details. Inspect any AI-generated descriptions for accuracy and clarity, and only use them if they align with what you would manually provide.
  
### Have a domain expert define the space
  * An effective space creator needs to understand the data and the insights that can be gleaned from it. Data analysts who are proficient in SQL typically have the knowledge and skills to curate the space.

### Define the purpose of your space
  * Identifying your space’s specific audience and purpose helps you decide which data, instructions, and test questions to use. A space should answer questions for a particular topic and audience, not general questions across various domains.

### Test and adjust
  * You should be your space’s first user. After you create a new space, start asking questions. Carefully examine the SQL generated in response to your questions. If Genie misinterprets the data, questions, or business jargon, you can intervene by editing the generated SQL or providing other specific instructions. Keep testing and editing until you’re getting reliable responses.

  * After you’ve reviewed a question, you can add it as a benchmark question that you can use to systematically test and score your space for overall accuracy. You can use variations and different questions phrasings to test Genie’s responses. See Use benchmarks in a Genie space.

  * See Troubleshooting for ideas on fixing erroneous responses.

### Conduct user testing
  * After verifying response quality through testing, recruit a business user to try the Genie space. Use the following guidelines to provide a smooth user journey and collect feedback for ongoing improvement:
    * Set expectations that their job is to help to refine the room.
    * Ask them to focus their testing on the specific topic and questions the space is designed to answer.
    * If they receive an incorrect response, encourage users to add additional instructions and clarifications in the chat to refine the answer. When a correct response is provided, they should upvote the final query to minimize similar errors in future interactions.
    * Tell users to upvote or downvote responses using the built-in feedback mechanism.
    * Invite users to share additional feedback and unresolved questions directly with the space authors. Authors and editors can use feedback to refine instructions, examples, and trusted assets.
  * Consider providing training materials or a written document with guidelines for testing the room and providing feedback. As business users test the space, you’ll see the questions they’ve asked on the History tab. Continue adding instructions to help Genie correctly interpret the questions and data to provide accurate answers. See Review history and feedback to learn more about how to monitor Genie spaces.